In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [2]:
import os 
from glob import glob
import random
import numpy as np

In [3]:
os.listdir("cropped_dataset/train")

['1', '2', '.ipynb_checkpoints', '0']

In [4]:
train_dir = "cropped_dataset/train" 
test_dir = "cropped_dataset/test"

# fit 데이터셋 만들기

In [5]:
train_path = glob(train_dir+"/*/*.jpg")
test_path = glob(test_dir+"/*/*.jpg")

In [6]:

random.seed(0)
random.shuffle(train_path)
random.shuffle(test_path)
print(train_path[0])
print(test_path[0])

cropped_dataset/train/1/aug_3_0_4612.jpg
cropped_dataset/test/0/aug_1_2_0_3803.jpg


In [7]:
def get_label(filepath):
    return filepath.split("/")[-2]

In [8]:
train_x = []
for file in train_path:
    gfile = tf.io.read_file(file)
    image = tf.io.decode_image(gfile)
    image = tf.image.resize(image, [50, 50])
    image = tf.image.rgb_to_grayscale(image)
    train_x.append(np.asarray(image))

In [9]:
train_y = [get_label(file) for file in train_path]

In [10]:
test_x = []
for file in test_path:
    gfile = tf.io.read_file(file)
    image = tf.io.decode_image(gfile)
    image = tf.image.resize(image, [50, 50])
    image = tf.image.rgb_to_grayscale(image)
    test_x.append(np.asarray(image))

In [11]:
test_y = [get_label(file) for file in test_path]

In [12]:
train_x[10].shape

(50, 50, 1)

In [13]:
type(train_x[0].shape)
print(type(np.array(train_x[0])))

<class 'numpy.ndarray'>


In [14]:
len(train_y)

13395

In [15]:
np.unique(train_y, return_counts=True)

(array(['0', '1', '2'], dtype='<U1'), array([4478, 4419, 4498]))

In [16]:
np.unique(test_y, return_counts=True)

(array(['0', '1', '2'], dtype='<U1'), array([496, 489, 498]))

In [17]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [18]:
train_x = train_x/255
test_x = test_x/255

In [19]:
train_x.shape

(13395, 50, 50, 1)

In [20]:
test_x.shape

(1483, 50, 50, 1)

In [21]:
train_y = train_y.astype("int32")
test_y = test_y.astype("int32")

# 모델 정의

In [22]:
input_shape = (50,50,1)
#learning_rate = 0.01
dropout_rate = 0.4
num_classes=3

In [23]:
from tensorflow.keras.models import Sequential

model = Sequential()
#inputs = layers.Input(input_shape)

# Feature Extraction
model.add(tf.keras.Input(shape=input_shape)) #Input 생략가능
model.add(layers.Conv2D(16, (3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(16, (3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(32, (3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(32, (3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))

#Classification
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(num_classes, activation="softmax"))

#주의: binary classification에서는 output dense가 1이다 
#net = layers.Activation("sigmoid")(net)

#model = tf.keras.Model(inputs=inputs, outputs=net, name="NN")

# Training

In [24]:
EPOCHS = 30

In [25]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        #labels = tf.reshape(labels, [32,1])
        #print(predictions.shape)
        #print(labels.shape)
        
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
    
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [26]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

In [27]:
#model = ConvNet()

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [29]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [30]:
class EarlyStopping:
    def __init__(self):
        self.test_acc_list = []
        self.length = 10
        
    def step(self, acc):
        self.test_acc_list.append(acc.numpy())
        if len(self.test_acc_list) > self.length:
            self.test_acc_list.pop(0)
    def is_stop(self, epoch):
        mean = sum(self.test_acc_list)/len(self.test_acc_list)
        recent_acc = self.test_acc_list[-1]
        if epoch > 10:
            
            if recent_acc - mean <=0.01:
                print("Early Stopping!")
                print("recent_acc: {}, mean acc: {} ".format(recent_acc, mean) )
                return True
            
        return False

In [31]:
es = EarlyStopping()

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
    
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(
  
        epoch+1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))
    
    
    es.step(test_accuracy.result())
    if es.is_stop(epoch):
        break
        
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    
    

Epoch 1, Loss: 0.42496415972709656, Accuracy: 81.0376968383789, Test Loss: 0.2640047073364258, Test Accuracy: 88.73904418945312
Epoch 2, Loss: 0.20618249475955963, Accuracy: 92.1612548828125, Test Loss: 0.20994219183921814, Test Accuracy: 91.90829467773438
Epoch 3, Loss: 0.14283955097198486, Accuracy: 94.5949935913086, Test Loss: 0.1471882313489914, Test Accuracy: 94.67295837402344
Epoch 4, Loss: 0.08391281217336655, Accuracy: 96.87943267822266, Test Loss: 0.127688929438591, Test Accuracy: 94.94268035888672
Epoch 5, Loss: 0.06239883229136467, Accuracy: 97.70809936523438, Test Loss: 0.13711614906787872, Test Accuracy: 95.68441772460938
Epoch 6, Loss: 0.0399547703564167, Accuracy: 98.65621185302734, Test Loss: 0.11536384373903275, Test Accuracy: 96.35873413085938
Epoch 7, Loss: 0.03325390815734863, Accuracy: 98.7756576538086, Test Loss: 0.1160014346241951, Test Accuracy: 96.69588470458984
Epoch 8, Loss: 0.032103218138217926, Accuracy: 98.86524963378906, Test Loss: 0.10445286333560944, Te

In [32]:
#model.save("model/", include_optimizer=True, save_format='tf')
model.save('model/basic-cnn-sequential-expert.h5')